In [ ]:
import os
import pandas as pd
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from rdkit import Chem
from rdkit.Chem import Draw, rdMolDescriptors
from docx.shared import Pt, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_LINE_SPACING
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.enum.table import WD_TABLE_ALIGNMENT
import datetime

def generar_plantilla_y_reportes():
    # Configuración de rutas
    csv_path = r"C:\Tareas002\SegundoParcial\BBBP.eje2.csv"  # Ajusta tu ruta
    output_dir = os.path.join(os.path.dirname(csv_path), 'informes_generados')
    os.makedirs(output_dir, exist_ok=True)

    # ========================================================================
    # PARTE 1: GENERAR PLANTILLA (ARCHIVO BASE)
    # ========================================================================
    doc_plantilla = Document()
    
    # --- Título principal ---
    titulo = doc_plantilla.add_paragraph()
    titulo.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run = titulo.add_run("ANÁLISIS DEL CONJUNTO DE DATOS DE PENETRACIÓN DE LA BARRERA HEMATOENCEFÁLICA (BBBP)")
    run.bold = True
    run.font.size = Pt(14)

    # --- Autor ---
    autor = doc_plantilla.add_paragraph()
    autor.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    autor.add_run("Elaborado por: [Nombre]").italic = True
    autor.paragraph_format.space_after = Pt(24)

    # --- Sección 1: Resumen ---
    doc_plantilla.add_paragraph("1. Resumen", style="Heading 1")
    doc_plantilla.add_paragraph("[Complete con un resumen del compuesto y su relevancia en BBBP.]")

    # --- Sección 2: Tabla de datos
    doc_plantilla.add_paragraph("2. Identificación del compuesto", style="Heading 1")
    table = doc_plantilla.add_table(rows=4, cols=2)
    table.style = "Table Grid"
    data_plantilla = [
        ("Número de Compuesto", "[Ej: 001]"),
        ("Nombre", "[Ej: Dopamina]"),
        ("Fórmula Molecular", "[Ej: C8H11NO2]"),
        ("Estado BBBP", "[Sí/No penetra]"),
    ]
    for idx, (label, value) in enumerate(data_plantilla):
        table.cell(idx, 0).text = label
        table.cell(idx, 1).text = value
        table.cell(idx, 0).paragraphs[0].runs[0].font.bold = True

    # --- Sección 3: Estructura Molecular ---
    doc_plantilla.add_paragraph("3. Estructura Molecular", style="Heading 1")
    doc_plantilla.add_paragraph("[Inserte imagen de la estructura aquí]").alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Guardar plantilla
    plantilla_path = os.path.join(output_dir, "Plantilla_BBBP.docx")
    doc_plantilla.save(plantilla_path)
    print(f"✓ Plantilla generada: {plantilla_path}")

def generar_informes_bbbp():
    
    csv_path = r"C:\Tareas002\SegundoParcial\BBBP.eje2.csv"
    output_dir = os.path.join(os.path.dirname(csv_path), 'informes_generados')
    num_reports = 200 ################################

    print("Iniciando generación de informes...")
    
    if not os.path.exists(csv_path):
        print(f"✗ Error: No se encuentra el archivo CSV en {csv_path}")
        return

    os.makedirs(output_dir, exist_ok=True)

    try:
        df = pd.read_csv(csv_path)
        print(f"• Se cargaron {len(df)} registros del CSV")

        generated_count = 0
        for i, row in df.head(num_reports).iterrows():
            try:
                doc = Document()

                # Título principal
                titulo = doc.add_paragraph()
                titulo.alignment = WD_ALIGN_PARAGRAPH.CENTER
                run = titulo.add_run("ANÁLISIS DEL CONJUNTO DE DATOS DE PENETRACIÓN DE LA BARRERA HEMATOENCEFÁLICA (BBBP)")
                run.bold = True

                # Autor
                autor = doc.add_paragraph()
                autor.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                autor.add_run("Elaborado por: Anabel Plata").italic = True
                autor.paragraph_format.space_after = Pt(24)

                # ============================================================
                # SECCIÓN 1: RESUMEN
                # ============================================================
                heading = doc.add_paragraph()
                heading.paragraph_format.first_line_indent = Inches(0.5)
                heading.paragraph_format.left_indent = Pt(0)
                heading.paragraph_format.right_indent = Pt(0)
                heading.paragraph_format.space_after = Pt(6)
                heading.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                run_heading = heading.add_run('1. Resumen')
                run_heading.bold = True
                run_heading.font.name = 'Times New Roman'
                run_heading.font.size = Pt(12)

                # Contenido del resumen
                resumen_text = (
                    f"El compuesto {row['name']} {'PENETRA' if row['p_np'] == 1 else 'NO PENETRA'} "
                    f"la barrera hematoencefálica (BBB). "
                    f"{'Posible uso en terapias neurológicas.' if row['p_np'] == 1 else 'No efectivo para SNC sin modificación.'} "
                    f"(Modelo predictivo: Martins et al., 2012)"
                        )

                resumen_para = doc.add_paragraph()
                resumen_para.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
                resumen_para.paragraph_format.space_after = Pt(6)
                resumen_para.paragraph_format.first_line_indent = Pt(0)
                resumen_para.paragraph_format.left_indent = Pt(0)
                resumen_para.paragraph_format.right_indent = Pt(0)
                resumen_para.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                run_resumen = resumen_para.add_run(resumen_text)
                run_resumen.font.name = 'Tahoma'
                run_resumen.font.size = Pt(10)

                # ============================================================
                # SECCIÓN 2: INTRODUCCIÓN
                # ============================================================
                heading = doc.add_paragraph()
                heading.paragraph_format.first_line_indent = Inches(0.5)
                heading.paragraph_format.left_indent = Pt(0)
                heading.paragraph_format.right_indent = Pt(0)
                heading.paragraph_format.space_after = Pt(6)
                heading.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                run_heading = heading.add_run('2. Introducción')
                run_heading.bold = True
                run_heading.font.name = 'Times New Roman'
                run_heading.font.size = Pt(12)

                intro = [
                    "La Barrera Hematoencefálica (BHE) es una estructura selectiva que controla el paso de sustancias desde la sangre al cerebro, dificultando la entrega de fármacos al sistema nervioso central (SNC). El conjunto de datos BBBP permite evaluar la capacidad de diversos compuestos para atravesar esta barrera, siendo fundamental para el desarrollo de terapias neurológicas.\n"
                ]

                for texto in intro:  
                    if texto.strip():  
                        para = doc.add_paragraph()
                        para.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
                        para.paragraph_format.space_after = Pt(6)
                        para.paragraph_format.first_line_indent = Pt(0)
                        para.paragraph_format.left_indent = Pt(0)
                        para.paragraph_format.right_indent = Pt(0)
                        para.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                        run = para.add_run(texto) 
                        run.font.name = 'Tahoma'
                        run.font.size = Pt(10)

                # ============================================================
                # SECCIÓN 3: IDENTIFICACIÓN DEL COMPUESTO
                # ============================================================
                heading = doc.add_paragraph()
                heading.paragraph_format.first_line_indent = Inches(0.5)
                heading.paragraph_format.left_indent = Pt(0)
                heading.paragraph_format.right_indent = Pt(0)
                heading.paragraph_format.space_after = Pt(6)
                heading.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                run_heading = heading.add_run('3. Identificación del compuesto')
                run_heading.bold = True
                run_heading.font.name = 'Times New Roman'
                run_heading.font.size = Pt(12)

                # Crear tabla de información
                mol = Chem.MolFromSmiles(row['smiles']) if pd.notna(row['smiles']) else None
                data_rows = [
                    ("Número de Compuesto", str(i+1)),
                    ("Nombre", str(row['name'])),
                    ("Fórmula Molecular", rdMolDescriptors.CalcMolFormula(mol) if mol else "N/A"),
                    ("Estado BBBP", "Sí penetra, podría ser un buen candidato para tratar enfermedades del SNC" if row['p_np'] == 1 else "No penetra, puede no ser útil para terapias dirigidas al cerebro"),
                ]

                table = doc.add_table(rows=len(data_rows), cols=2)
                table.style = 'Table Grid'
                table.autofit = False
                table.columns[0].width = Inches(1.8)
                table.columns[1].width = Inches(3.2)
                table.alignment = WD_TABLE_ALIGNMENT.CENTER
                
                for idx, (label, value) in enumerate(data_rows):
                    table.cell(idx, 0).text = label
                    table.cell(idx, 1).text = str(value)
                    table.cell(idx, 0).paragraphs[0].runs[0].font.bold = True
                    table.cell(idx, 0).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
                    table.cell(idx, 1).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

                for r in table.rows:
                    for cell in r.cells:
                        cell.paragraphs[0].runs[0].font.size = Pt(11)
                        cell.paragraphs[0].runs[0].font.name = 'Times New Roman'
                        cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER  
                        cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER  
                        cell.margin_top = Inches(0.05)
                        cell.margin_bottom = Inches(0.05)
                        cell.margin_left = Inches(0.1)
                        cell.margin_right = Inches(0.1)

                # ============================================================
                # SECCIÓN 4: ESTRUCTURA MOLECULAR
                # ============================================================
                mol = Chem.MolFromSmiles(row['smiles']) 
                if mol:
                    img_path = os.path.join(output_dir, f"temp_{i}.png")
                    Draw.MolToFile(mol, img_path, size=(300, 300))
                    
                    space_paragraph = doc.add_paragraph()
                    space_paragraph.paragraph_format.space_after = Pt(12) 
                    img_paragraph = doc.add_paragraph()
                    img_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  
                    
                    run_img = img_paragraph.add_run()
                    run_img.add_picture(img_path, width=Inches(2.5))
                    os.remove(img_path)
                    
                    heading = doc.add_paragraph()
                    heading.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  
                    heading.paragraph_format.space_after = Pt(6)  
                    
                    run_heading = heading.add_run('4. Estructura Molecular')
                    run_heading.bold = True
                    run_heading.font.name = 'Times New Roman'
                    run_heading.font.size = Pt(12)
                else:
                    no_structure = doc.add_paragraph("(Estructura molecular no disponible)")
                    no_structure.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

                # ============================================================
                # SECCIÓN 5: CONCLUSIONES
                # ============================================================
                heading = doc.add_paragraph()
                heading.paragraph_format.first_line_indent = Inches(0.5)
                heading.paragraph_format.left_indent = Pt(0)
                heading.paragraph_format.right_indent = Pt(0)
                heading.paragraph_format.space_after = Pt(6)
                heading.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                run_heading = heading.add_run('5. Conclusiones')
                run_heading.bold = True
                run_heading.font.name = 'Times New Roman'
                run_heading.font.size = Pt(12)

                # Contenido de conclusiones
                conclusiones_text = (
                    f"En base al análisis realizado, el compuesto {row['name']} {'presenta características adecuadas para '
                    'ser considerado en el desarrollo de fármacos dirigidos al sistema nervioso central, dada su capacidad '
                    'para atravesar la barrera hematoencefálica.' if row['p_np'] == 1 else 'no muestra capacidad para penetrar '
                    'la barrera hematoencefálica en su forma actual, por lo que requeriría modificaciones estructurales para '
                    'su potencial uso en terapias neurológicas.'}\n\n"
                    "Recomendaciones:\n"
                    f"- {'Priorizar estudios preclínicos para evaluar su actividad farmacológica en el SNC.' if row['p_np'] == 1 else 'Explorar derivados estructurales que mejoren su permeabilidad BBB.'}\n"
                    "- Validar estos resultados predictivos con estudios in vitro o in vivo.\n"
                    "- Considerar propiedades adicionales como toxicidad y metabolismo."
                )

                conclusiones_para = doc.add_paragraph()
                conclusiones_para.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
                conclusiones_para.paragraph_format.space_after = Pt(6)
                conclusiones_para.paragraph_format.first_line_indent = Pt(0)
                conclusiones_para.paragraph_format.left_indent = Pt(0)
                conclusiones_para.paragraph_format.right_indent = Pt(0)
                conclusiones_para.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                run_conclusiones = conclusiones_para.add_run(conclusiones_text)
                run_conclusiones.font.name = 'Tahoma'
                run_conclusiones.font.size = Pt(10)

                # ============================================================
                # SECCIÓN 6: BIBLIOGRAFÍA 
                # ============================================================
                heading = doc.add_paragraph()
                heading.paragraph_format.first_line_indent = Inches(0.5)
                heading.paragraph_format.left_indent = Pt(0)
                heading.paragraph_format.right_indent = Pt(0)
                heading.paragraph_format.space_after = Pt(6)
                heading.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
                run_heading = heading.add_run('6. Bibliografía')
                run_heading.bold = True
                run_heading.font.name = 'Times New Roman'
                run_heading.font.size = Pt(12)

                bibliografia = doc.add_paragraph()
                bibliografia.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
                bibliografia.paragraph_format.space_after = Pt(6)
                bibliografia.add_run("1. Martins, I. F., et al. (2012). 'A Bayesian approach to in silico blood-brain barrier penetration modeling.' Journal of Chemical Information and Modeling, 52(6), 1686-1697.").font.size = Pt(10)
                bibliografia.add_run("\n2. Lipinski, C. A., et al. (2001). 'Experimental and computational approaches to estimate solubility and permeability in drug discovery and development settings.' Advanced Drug Delivery Reviews, 46(1-3), 3-26.").font.size = Pt(10)

                # Guardar documento
                report_name = f"Reporte_BBBP_{i+1:03d}.docx"
                doc.save(os.path.join(output_dir, report_name))
                generated_count += 1
                
            except Exception as e:
                print(f"✗ Error con compuesto {i+1}: {str(e)}")
                continue

        print(f"\n✓ Informes generados correctamente: {generated_count}/{num_reports}")
        print(f"• Carpeta de salida: {output_dir}")

    except Exception as e:
        print(f"✗ Error crítico: {str(e)}")

# Ejecutar
generar_plantilla_y_reportes()
generar_informes_bbbp()

✓ Plantilla generada: C:\Tareas002\SegundoParcial\informes_generados\Plantilla_BBBP.docx
Iniciando generación de informes...
• Se cargaron 2050 registros del CSV

✓ Informes generados correctamente: 11/11
• Carpeta de salida: C:\Tareas002\SegundoParcial\informes_generados
